<a href="https://colab.research.google.com/github/krutse/sim-git/blob/master/python/Ch14/krut-se_%D0%9A%D0%B5%D0%B9%D1%81_%D0%9B%D0%BE%D0%B3%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-07-29 06:40:03--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-07-29 06:40:03--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.04s   

2025-07-29 06:40:03 (9.98

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

INFO  | 2022-11-26 00:01:03,024  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-26, количество людей с автопродлением подписки: 0

INFO  | 2022-11-26 00:01:03,027  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-27 00:01:02,455  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-27 00:01:02,480  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-27, количество людей с автопродлением подписки: 0

INFO  | 2022-11-27 00:01:02,483  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-28 00:01:02,406  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-28 00:01:02,424  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-28, количество людей с автопродлением подписки: 0

INFO  | 2022-11-28 00:01:02,427  | f

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:

def count_success_and_failure(file_path):
  i=0
  e=0
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()
    for line in lines:
      r = [x.strip() for x in line.split('|')]
      if r[0] == 'INFO' and r[4].find('Обновляем подписку') > -1:
        i += 1
      elif r[0] == 'ERROR':
        e += 1
  return ((i-e, e))


count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [ ]:
from datetime import datetime, timedelta
def auto_renewal_sub(log_file_path):
  subscr = set()
  res = {}
  with open(log_file_path, 'r', encoding = 'utf-8') as f, open('auto_renewal_sub.txt','w', encoding = 'utf-8') as w:
    lines = f.readlines()
    for line in lines:
      r = [x.strip() for x in line.split('|')]
      if r[0] == 'INFO' and r[4].find('Cегодня') > -1 :
        subscr.add((datetime.fromisoformat(r[1]).strftime('%Y-%m-%d'), (r[4].split(',')[1]).split(':')[1].strip()))
    mindt = datetime.strptime(min(list(subscr))[0],'%Y-%m-%d')
    maxdt = datetime.strptime(max(list(subscr))[0] ,'%Y-%m-%d')
    ds = (maxdt - mindt).days + 1
    res = {(mindt + timedelta(days=i)).strftime('%Y-%m-%d'): 0 for i in range(ds)}
    for i in subscr:
      res[i[0]] = float(i[1])
    w.write(f'Среднее: {[round(sum(list(res.values())[:i+1])/(i+1),2) for i in range(len(res))]}\n')
    # w.write(f'Медиана: {[int(sorted(list(res.values()))[(i+1) //2]) for i in range(len(res))]}')
    w.write(f'Медиана: {[int(sorted(list(res.values())[:i+1],reverse=True)[(i+1) //2]) for i in range(len(res))]}')


auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2025-07-28 20:12:47--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-07-28 20:12:47--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt.1’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2025-07-28 20

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [ ]:
from datetime import datetime, date

def sub_renewal_by_day(file_path):
  lines = []
  errln = []
  weekday_names = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
  with open(file_path, 'r', encoding = 'utf-8') as f, open('weekdays.txt','w', encoding = 'utf-8') as w:
    for line in f:
      if 'Обновляем' in line and 'INFO' in line:
        lines.append(line.split('|'))
      elif 'ERROR' in line and 'ошибка' in line:
        errln.append(line.split('|'))
    subscr = list(((datetime.fromisoformat(r[1].strip()).strftime('%Y-%m-%d'), int((r[4].split(',')[0]).split(':')[1].strip())  ) for r in lines))
    errs = list(((datetime.fromisoformat(r[1].strip()).strftime('%Y-%m-%d'), int((r[4].split('-')[0]).split(':')[1].strip()) ) for r in errln))
    wk = dict.fromkeys(range(7), 0)
    er = dict.fromkeys(range(7), 0)
    for i in subscr:
      wk[date.fromisoformat(i[0]).weekday()] += 1
    for i in errs:
      er[date.fromisoformat(i[0]).weekday()] += 1
    res = ('\n').join([f'{weekday_names[k]}: {v - er[k]}' for k, v in wk.items()])
    w.write('Количество обновлений подписки по дням недели:\n')
    w.write(res)

sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2025-07-29 07:33:23--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2025-07-29 07:33:23--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt.1’

cor_weekdays.txt.1  100%[===================>]     238  --.-KB/s    in 0s      

2025-07-29 07:33:23 (4.41 MB/s) - ‘co

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
